<a href="https://colab.research.google.com/github/kmazurov/ML-model-for-a-metalworking-enterprise/blob/main/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D1%82%D0%BE%D0%BA%D1%81%D0%B8%D1%87%D0%BD%D1%8B%D1%85_%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D0%BD%D1%82%D0%B0%D1%80%D0%B8%D0%B5%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача: 
Обучить модель классифицировать комментарии на позитивные и негативные, основываясь на наборе данных с разметкой о токсичности правок.

Основная метрики качества *F1* должна быть не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
</a></span><ul class="toc-item"><li><span><a href="#1.-Подготовка" data-toc-modified-id="1.-Подготовка-1.1">1. Подготовка</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-1.1.1">Вывод</a></span></li></ul></li><li><span><a href="#2.-Обучение" data-toc-modified-id="2.-Обучение-1.2">2. Обучение</a></span></li><li><span><a href="#3.-Выводы" data-toc-modified-id="3.-Выводы-1.3">3. Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-1.4">Чек-лист проверки</a></span></li></ul></li></ul></div>

## 1. Подготовка

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [ ]:
display(data.head())

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [ ]:
display(data.isnull().sum())
display(data.duplicated().sum())

text     0
toxic    0
dtype: int64

0

In [ ]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

### Вывод

- В данных нет пропущенных значений или дубликатов 
- Примерное соотношение токсичных комментариев к обычным 9:1

**Для работы над задачей проведем лемматизацию, очистим от стоп-слов**

In [ ]:
data['lemm_text'] = data['text']

In [ ]:
# Очистка текста

def clean_text(row):
    row = re.sub(r"(?:\n|\r)", " ", row)
    row = re.sub(r"[^a-zA-Z ]+", "", row).strip()
    row = row.lower()
    
    return row

In [ ]:
data['lemm_text'] = data['lemm_text'].apply(clean_text)

display(data.head())

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,daww he matches this background colour im seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not trying to edit war its j...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cant make any real suggestions on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    word_list = nltk.word_tokenize(text)
    llemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
        
    return llemmatized_output

In [ ]:
data['lemm_text'] = data['lemm_text'].apply(lemmatize)

display(data.head())

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,daww he match this background colour im seemin...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man im really not trying to edit war it ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i cant make any real suggestion on improv...
4,"You, sir, are my hero. Any chance you remember...",0,you sir are my hero any chance you remember wh...


**TF IDF без стоп-слов**

In [ ]:
# Для удобства дальнейшей работы удалим столбец с исходным текстом

data = data.drop(['text'], axis=1)

In [ ]:
# Разделение датафрейма на обучающую и тестовую выборки

features = data['lemm_text']
target = data['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target, stratify=target, 
                                                                            test_size=0.25)

corpus_train = features_train.values.astype('U')
corpus_test = features_test.values.astype('U')

In [ ]:
# Обработка стоп-слов

stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

# Учим и трансформируем обучающую выборку
features_train = count_tf_idf.fit_transform(corpus_train)

print("Размер обучающей выборки tf_idf с учётом стоп-слов:", features_train.shape)

# Трансформируем тестовую выборку
features_test = count_tf_idf.transform(corpus_test)

print("Размер тестовой выборки tf_idf с учётом стоп-слов:", features_test.shape)

Размер обучающей выборки tf_idf с учётом стоп-слов: (119678, 175621)
Размер тестовой выборки tf_idf с учётом стоп-слов: (39893, 175621)


## 2. Обучение

**Основной моделью для обучения выберем логистическую регрессию. Эта модель лучше всего подходит для работы с текстами. Для сравнения возьмем модель Случайного леса. В модели логистической регрессии поработаем с коэффициент регуляризации С.**

**Логистическая регрессия TF IDF**

In [ ]:
# Модель со стандартными гиперпараметрами

model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predict = model.predict(features_test)
print('Оценка F1 для модель логистической регрессии:', f1_score(target_test, predict))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Оценка F1 для модель логистической регрессии: 0.7313052600029971


In [ ]:
model = LogisticRegression(C=10, random_state=12345)
model.fit(features_train, target_train)
predict = model.predict(features_test)
print('Оценка F1 для модель логистической регрессии:', f1_score(target_test, predict))

Оценка F1 для модель логистической регрессии: 0.7811419881100512


**Случайный лес TF IDF**

In [ ]:
# Модель со стандартными гиперпараметрами

model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predict = model.predict(features_test)

print('Оценка F1 для модель случайного леса:', f1_score(target_test, predict))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Оценка F1 для модель случайного леса: 0.6670842467898528


In [ ]:
model = RandomForestClassifier(random_state=12345, n_estimators=500, max_leaf_nodes=3)
model.fit(features_train, target_train)
predict = model.predict(features_test)
print(f1_score(target_test, predict))

0.0


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


**Логистическая регрессия Мешок слов**

In [ ]:
# Обработка стоп-слов

stopwords = set(nltk_stopwords.words('english'))
count_vect = CountVectorizer(stop_words=stopwords)

# Учим и трансформируем обучающую выборку
features_train = count_vect.fit_transform(corpus_train)

print("Размер обучающей выборки мешка слов с учётом стоп-слов:", features_train.shape)

# Трансформируем тестовую выборку
features_test = count_vect.transform(corpus_test)

print("Размер тестовой выборки мешка слов с учётом стоп-слов:", features_test.shape)

Размер обучающей выборки мешка слов с учётом стоп-слов: (119678, 175621)
Размер тестовой выборки мешка слов с учётом стоп-слов: (39893, 175621)


In [ ]:
# Модель со стандартными гиперпараметрами

model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predict = model.predict(features_test)
print('Оценка F1 для модель логистической регрессии:', f1_score(target_test, predict))

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Оценка F1 для модель логистической регрессии: 0.7025972076264826


/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [ ]:
model = LogisticRegression(C=10, random_state=12345)
model.fit(features_train, target_train)
predict = model.predict(features_test)
print('Оценка F1 для модель логистической регрессии:', f1_score(target_test, predict))

Оценка F1 для модель логистической регрессии: 0.7059877557115125


**Случайный лес мешок слов**

In [ ]:
# Модель со стандартными гиперпараметрами

model = RandomForestClassifier(random_state=12345)
model.fit(features_train, target_train)
predict = model.predict(features_test)

print('Оценка F1 для модель случайного леса:', f1_score(target_test, predict))

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Оценка F1 для модель случайного леса: 0.6687306501547987


## 3. Выводы

1. Лучший результат показала модель логистической регрессии, с гиперпараметром С=10 и TF IDF
2. Модель случайного леса показала результат только на гиперпараметрах по умолчанию
3. Метод преобразования текстов TF IDF эфективнее, чем создание мешка слов
4. Для эффективной предобработки текста необходима провести чистку, а после лемматизацию